In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep 28 05:55:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [8]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 2.2 MB/s 


In [36]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain

from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score


#https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 参考
from skmultilearn.problem_transform import BinaryRelevance
# from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN


In [10]:
train_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/train.json"
test_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/test.json"

with open(train_file_path, 'r') as f:
    data = json.load(f)
df_train = pd.DataFrame(data)

with open(test_file_path, 'r') as f:
    data = json.load(f)
df_test = pd.DataFrame(data)

In [11]:
# Retrive the non-prolific author and prolific author index
non_prolific_author_index = []
prolific_author_index = []
for i,j in enumerate(df_train['authors']):
  sum = 0
  for k in j:
    if k < 100 and k >= 0:
      sum += 1
  if sum == 0:
    non_prolific_author_index.append(i)
  else:
    prolific_author_index.append(i)

df_non_prolifc_authors = df_train.drop(prolific_author_index)
df_prolific_authors = df_train.drop(non_prolific_author_index)

In [12]:
df_prolific_authors

,authors,year,abstract,venue,title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [13]:
df_test

,identifier,coauthors,year,abstract,venue,title
0,0,"[16336, 1762, 4357, 12564]",19,"[37, 1662, 3207, 10, 33, 2037, 1738, 1642, 155...",223,"[3207, 24, 1798, 1738, 37, 2375, 1568, 11, 53,..."
1,1,"[21189, 14088]",19,"[1731, 2130, 3674, 1705, 1656, 3077, 1546, 367...",223,"[40, 1560, 1536, 1544, 1609, 1705, 1658, 1543,..."
2,2,"[3625, 1198, 19889, 794, 2749, 7801]",19,"[1551, 1728, 3920, 1542, 1535, 1656, 1543, 153...",7,"[47, 1574, 1729, 1641, 11, 37, 2533, 2015, 47,..."
3,3,"[19810, 15173, 5876, 111]",19,"[51, 1535, 2115, 1543, 1811, 1700, 1657, 1684,...",21,"[1770, 53, 2054, 1549, 1529, 1723, 2796, 1547,..."
4,4,"[10932, 7668, 11907, 19601, 15307, 10492, 1049...",19,"[1775, 1746, 1842, 1525, 33, 2551, 1882, 1542,...",,"[18, 1924, 23, 1544, 3927, 2686, 1543, 1535, 1..."
...,...,...,...,...,...,...
795,795,"[2964, 9493]",19,"[1657, 1533, 1681, 1527, 1667, 2295, 1916, 236...",223,"[2796, 1547, 1531, 1550, 2370, 51, 1549, 2600,..."
796,796,"[6305, 4866, 5741, 15334]",19,"[1618, 1552, 1630, 2367, 1613, 3600, 1857, 155...",0,"[37, 1747, 2188, 11, 3623, 57, 2103, 1555, 37,..."
797,797,"[4856, 5063]",19,"[37, 1662, 33, 2095, 1726, 2090, 1553, 2410, 2...",0,"[1621, 11, 1560, 1816, 2151, 1533, 1534, 37, 1..."
798,798,"[18095, 2884, 15373]",19,"[46, 2337, 11, 2116, 2185, 1839, 3154, 3057, 1...",460,"[1832, 1552, 10, 3585, 10, 35, 1889, 1597, 425..."


In [14]:
# Retrieve the prolific author as label
prolific_author_label = []
non_prolific_author_label = []
for i in df_prolific_authors['authors']:
  prolific_author_label_each_instance = []
  non_prolific_author_label_each_instance = []
  for j in i:
    if j >= 0 and j < 100:
      prolific_author_label_each_instance.append(j)
    else:
      non_prolific_author_label_each_instance.append(j)
  prolific_author_label.append(prolific_author_label_each_instance)
  non_prolific_author_label.append(non_prolific_author_label_each_instance)


df_prolific_authors['coauthors'] = non_prolific_author_label

In [15]:
#One-hot encoding the labels
multilable_binarizer = MultiLabelBinarizer()
ohe_prolific_labels = multilable_binarizer.fit_transform(prolific_author_label).astype('float')

In [16]:
df_prolific_authors['sentences'] = df_prolific_authors['title'] + df_prolific_authors['abstract'] 
df_prolific_authors = df_prolific_authors.drop(['abstract','title'],axis = 1)

df_test['sentences'] = df_test['title'] + df_test['abstract'] 
df_test = df_test.drop(['abstract','title'],axis = 1)

In [17]:
df_prolific_authors

,authors,year,venue,coauthors,sentences
0,"[42, 13720, 36]",9,20,[13720],"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,2,"[1359, 15881]","[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,4,[],"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,9,[19617],"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,0,"[9641, 5623]","[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,23,"[19974, 8903]","[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,10,"[6353, 3037, 15856, 13521]","[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,31,"[7686, 3810, 7451]","[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,0,[14864],"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [18]:
# coauthors_ohe = multilable_binarizer.fit_transform(df_prolific_authors['coauthors']).astype('float')
# sentences_ohe = multilable_binarizer.fit_transform(df_prolific_authors['sentences']).astype('float')
# df_prolific_authors['coauthors'])

def convert_to_string(df):
  total_list = []
  for i in df:
    total_list.append(str(i)[1:-1].replace(",", ""))
  return total_list




# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

string_sentences = convert_to_string(df_prolific_authors['sentences'])
sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
sentences_counts = sentences_vectorizer.fit_transform(string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

string_coauthor = convert_to_string(df_prolific_authors['coauthors'])
coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
coauthor_counts = coauthor_vectorizer.fit_transform(string_coauthor).toarray()





In [19]:

coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_coauthors_features = pd.DataFrame(coauthor_counts)
df_coauthors_features = df_coauthors_features.astype('float')
df_coauthors_features.columns = coauthors_features_name

sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_sentence_features = pd.DataFrame(sentences_counts)
df_sentence_features = df_sentence_features.astype('float')
df_sentence_features.columns = sentence_features_name


df_merge_coauthors_sentences = pd.concat([df_coauthors_features, df_sentence_features], axis = 1)

In [20]:
df_merge_coauthors_sentences['coauthors_13720']

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7455    0.0
7456    0.0
7457    0.0
7458    0.0
7459    0.0
Name: coauthors_13720, Length: 7460, dtype: float64

In [21]:
X_train,X_test,y_train,y_test = train_test_split(df_merge_coauthors_sentences,ohe_prolific_labels,test_size=0.2,random_state=90051)

In [40]:
base_lr = LogisticRegression(solver='lbfgs', random_state=0)
chain = ClassifierChain(base_lr, order='random', random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
chain.fit(X_scaled_train, y_train)
predictions = chain.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)


array([0.72727273, 0.33333333, 0.54545455, 0.5       , 0.73684211,
       0.625     , 0.33333333, 0.81481481, 0.5       , 0.57142857,
       0.74576271, 0.55172414, 0.51162791, 0.37037037, 0.77777778,
       0.65306122, 0.68      , 0.57142857, 0.66666667, 0.84444444,
       0.57142857, 0.51612903, 0.5       , 0.52631579, 0.51612903,
       0.64      , 0.71428571, 0.74074074, 0.6       , 0.81818182,
       0.36363636, 0.66666667, 0.79166667, 0.77419355, 0.66666667,
       0.72727273, 0.66666667, 0.60869565, 0.64516129, 0.66666667,
       0.75      , 0.57894737, 0.66666667, 0.58823529, 0.52631579,
       0.8372093 , 0.53846154, 0.75      , 0.5       , 0.61538462,
       0.4516129 , 0.28571429, 0.14285714, 0.61818182, 0.52631579,
       0.66666667, 0.14285714, 0.76190476, 0.69565217, 0.62068966,
       0.5       , 0.69230769, 0.22222222, 0.7826087 , 0.42105263,
       0.63414634, 0.73684211, 0.70588235, 0.72727273, 0.66666667,
       0.76470588, 0.5       , 0.75      , 0.54545455, 0.6875 

In [41]:
f1_score(y_test, predictions, average='samples')

0.5102993744414656

In [42]:
#Test
# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

test_string_sentences = convert_to_string(df_test['sentences'])
test_sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
test_sentences_counts = test_sentences_vectorizer.fit_transform(test_string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

test_string_coauthor = convert_to_string(df_test['coauthors'])
test_coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
test_coauthor_counts = coauthor_vectorizer.fit_transform(test_string_coauthor).toarray()



test_coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_test_coauthors_features = pd.DataFrame(test_coauthor_counts)
df_test_coauthors_features = df_test_coauthors_features.astype('float')
df_test_coauthors_features.columns = test_coauthors_features_name

test_sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_test_sentence_features = pd.DataFrame(test_sentences_counts)
df_test_sentence_features = df_test_sentence_features.astype('float')
df_test_sentence_features.columns = test_sentence_features_name


df_test_merge_coauthors_sentences = pd.concat([df_test_coauthors_features, df_test_sentence_features], axis = 1)




In [43]:
df_test_merge_coauthors_sentences

,coauthors_0,coauthors_1,coauthors_2,coauthors_3,coauthors_4,coauthors_5,coauthors_6,coauthors_7,coauthors_8,coauthors_9,...,sentences_4990,sentences_4991,sentences_4992,sentences_4993,sentences_4994,sentences_4995,sentences_4996,sentences_4997,sentences_4998,sentences_4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
X_scaled_predict= scaler.transform(df_test_merge_coauthors_sentences)

In [45]:
predition = chain.predict(X_scaled_predict)

In [47]:
predition

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
predicted_list = [i.tolist() for i in predition]
total_index_list = []
for i in predicted_list:
  index_list = []
  for j,k in enumerate(i):
    if k == 1:
      index_list.append(str(j))
  if len(index_list) != 0:
    total_index_list.append(index_list)
  else:
    total_index_list.append([str(-1)])

predicted_str = []
for i in total_index_list:
  a = ','.join(i)
  predicted_str.append(a)

In [49]:

df_output = pd.DataFrame({'ID': list(range(800)),'Predict':predicted_str})
df_output.to_csv('predicted_result.csv',index = False)